In [1]:
%%configure -f

{"executorMemory": "2G","driverMemory":"1G","executorCores": 1,"numExecutors": 3,

"conf": {"spark.dynamicAllocation.enabled": "false",

          "spark.sql.parquet.enableVectorizedReader": "false",

          "spark.python.use.daemon": "false",

          "spark.pyspark.python": "python3",

          "spark.pyspark.virtualenv.enabled": "true",

          "spark.pyspark.virtualenv.type": "native",

          "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv"}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
91,application_1713221646413_0092,pyspark,idle,Link,Link,None,
99,application_1713221646413_0100,pyspark,idle,Link,Link,None,
103,application_1713221646413_0104,pyspark,idle,Link,Link,None,
108,application_1713221646413_0109,pyspark,shutting_down,Link,Link,None,


In [2]:
DM_df = spark.read.format("parquet").load("s3://msbx5420-spr24/teams/Group6/Digital_Music.parquet")
DM_df.show()

num_rows = DM_df.count()
num_columns = len(DM_df.columns)

print("Number of Rows:", num_rows)
print("Number of Columns:", num_columns)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
109,application_1713221646413_0110,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+------+-----------+------+--------------------+-------------+--------------------+--------------------+-----------------+
|      asin|helpful_vote|images|parent_asin|rating|                text|    timestamp|               title|             user_id|verified_purchase|
+----------+------------+------+-----------+------+--------------------+-------------+--------------------+--------------------+-----------------+
|B004RQ2IRG|           0|    []| B004RQ2IRG|   5.0|If i had a dollar...|1618972613292|                Nice|AFUOYIZBU3MTBOLYK...|             true|
|B0026UZEI0|           0|    []| B0026UZEI0|   5.0|awesome sound - c...|1308167525000|           Excellent|AHGAOIZVODNHYMNCB...|             true|
|B0055JSYHC|           0|    []| B0055JSYHC|   5.0|This is a great c...|1615838793006|       Great service|AFGEM6BXCYHUILEOA...|             true|
|B000F9SMUQ|           0|    []| B000F9SMUQ|   1.0|These are not rea...|1405219741000|             No good|AH3OG6QD6ED

In [3]:
sc.install_pypi_package('nltk')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
import os

def analyze_sentiment(text):
    if not os.path.isdir('./nltk_data/sentiment/vader_lexicon.zip'):
        nltk.download('vader_lexicon', download_dir='./nltk_data')
    nltk.data.path.append('./nltk_data')
    sid = SentimentIntensityAnalyzer()
    scores = sid.polarity_scores(text)
    return scores['compound']

analyze_sentiment_udf = udf(analyze_sentiment, DoubleType())
df_with_sentiment = DM_df.withColumn('sentiment_score', analyze_sentiment_udf(DM_df['text']))
df_with_sentiment.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+------+-----------+------+--------------------+-------------+--------------------+--------------------+-----------------+---------------+
|      asin|helpful_vote|images|parent_asin|rating|                text|    timestamp|               title|             user_id|verified_purchase|sentiment_score|
+----------+------------+------+-----------+------+--------------------+-------------+--------------------+--------------------+-----------------+---------------+
|B004RQ2IRG|           0|    []| B004RQ2IRG|   5.0|If i had a dollar...|1618972613292|                Nice|AFUOYIZBU3MTBOLYK...|             true|         0.9413|
|B0026UZEI0|           0|    []| B0026UZEI0|   5.0|awesome sound - c...|1308167525000|           Excellent|AHGAOIZVODNHYMNCB...|             true|         0.5848|
|B0055JSYHC|           0|    []| B0055JSYHC|   5.0|This is a great c...|1615838793006|       Great service|AFGEM6BXCYHUILEOA...|             true|         0.8779|
|B000F9SMUQ|          